<a href="https://colab.research.google.com/github/LeoneFabio/Egocentric-Vision/blob/main/Video_LLaVa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/PKU-YuanGroup/Video-LLaVA
!cd Video-LLaVA
!conda create -n videollava python=3.10 -y
!conda activate videollava
!pip install --upgrade pip  # enable PEP 660 support
!pip install -e .
!pip install -e ".[train]"
!pip install flash-attn --no-build-isolation
!pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d

Cloning into 'Video-LLaVA'...
remote: Enumerating objects: 973, done.
remote: Counting objects: 100% (491/491), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 973 (delta 400), reused 293 (delta 293), pack-reused 482 (from 3)
Receiving objects: 100% (973/973), 115.34 MiB | 26.52 MiB/s, done.
Resolving deltas: 100% (532/532), done.
/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.8 MB

In [ ]:
!pip install -q av
!pip install numpy
!pip install pillow
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 45.6 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-6_mugtb9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-6_mugtb9
  Resolved https://github.com/huggingface/transformers.git to commit 0e0516c119a3d1a2342d09166e583bb6fa647c8a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10322147 sha256=d2b0607ebe06b6f3264aaf6779366321bebf6f0bc81d9c8e7234a7316adad337
  Stored in directory: /tmp/pip-ephem-wheel-cache-x0ym_eya/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:

In [ ]:
from PIL import Image
import numpy as np
import av
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration, BitsAndBytesConfig
from huggingface_hub import hf_hub_download
import torch
import os
torch.cuda.is_available()

True

In [ ]:

def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


In [ ]:

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = VideoLlavaForConditionalGeneration.from_pretrained(
    "LanguageBind/Video-LLaVA-7B-hf",
   quantization_config=quantization_config,
    device_map="auto"
)

processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/66.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

In [ ]:
import os
import json
import re

def order_videos_by_clip_uid_and_prediction(json_file_path, video_folder_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    queries_info = [
        {"clip_uid": query["clip_uid"], "best_prediction": query["best_prediction"]}
        for query in data.get("top_50_queries", [])
    ]
    video_files = [f for f in os.listdir(video_folder_path) if f.endswith('.mp4')]

    video_mapping = {}
    for video in video_files:
        video_name = os.path.splitext(video)[0]

        video_mapping[video_name] = os.path.join(video_folder_path, video)

    ordered_videos = []
    for query_info in queries_info:

        best_prediction_int = [int(val) for val in query_info["best_prediction"]]
        combined_name = f"{query_info['clip_uid']}_{best_prediction_int[0]}_{best_prediction_int[1]}"

        if combined_name in video_mapping:
            ordered_videos.append(video_mapping[combined_name])
        else:
            ordered_videos.append(None)
    return ordered_videos

json_file_path = "/content/top_50_queries.json"
video_folder_path = "/content/cut_clips"

ordered_videos = order_videos_by_clip_uid_and_prediction(json_file_path, video_folder_path)
print(ordered_videos)


['/content/cut_clips/5726971c-b3cc-43ed-8071-f6ee143e417d_0_14.mp4', '/content/cut_clips/5d531ac1-010a-4e67-ba1a-96e485b14968_0_37.mp4', '/content/cut_clips/1731de62-b1b9-4b84-bd55-412cd67e9b3c_0_18.mp4', '/content/cut_clips/9093c341-5f95-4402-a2dd-6a84aa42bf1f_0_45.mp4', '/content/cut_clips/ab094ea2-9251-4f10-945b-c2ab00c5282e_0_7.mp4', '/content/cut_clips/b8654118-84a4-4167-83c9-f268cc15f7b2_0_26.mp4', '/content/cut_clips/75d3fc52-3776-47d4-b7fd-8074d30b06d1_131_180.mp4', '/content/cut_clips/4ba774a8-cd2a-4889-9971-cc91f5c1afd4_11_15.mp4', '/content/cut_clips/1731de62-b1b9-4b84-bd55-412cd67e9b3c_41_56.mp4', '/content/cut_clips/88dcb32f-a537-47de-b3bf-f9149352bbb9_232_242.mp4', '/content/cut_clips/3672773c-6ff8-47c2-9ef9-bb00c65814ef_0_67.mp4', '/content/cut_clips/cab983c1-d36e-4afa-8116-1e2bde4a4a4c_101_116.mp4', '/content/cut_clips/2237fc47-e8c9-4751-9b02-6189913b4b4d_0_3.mp4', '/content/cut_clips/49931037-b822-4c7b-baf4-4626c1e6b6ea_41_153.mp4', '/content/cut_clips/96453857-2454-41

In [ ]:
clips = []
for video in ordered_videos:
  container = av.open(video)
  # sample uniformly 8 frames from the video (we can sample more for longer videos)
  total_frames = container.streams.video[0].frames
  indices = np.arange(0, total_frames, total_frames / 8).astype(int)
  clips.append(read_video_pyav(container, indices))



In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# np array with shape (frames, height, width, channels)
video = clips[0]

fig = plt.figure()
im = plt.imshow(video[0,:,:,:])

plt.close() # this is required to not display the generated image

def init():
    im.set_data(video[0,:,:,:])

def animate(i):
    im.set_data(video[i,:,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],interval=100)
HTML(anim.to_html5_video())

In [ ]:
import json

json_file_path = '/content/top_50_queries.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)


prompts = []
for query in data.get("top_50_queries", []):
  conversation = f"USER: <video> {query['query']} ASSISTANT:"

  prompts.append(conversation)



In [ ]:
print(prompts)

['USER: <video> Where is the soap? ASSISTANT:', 'USER: <video> What vegetables did I chop?  ASSISTANT:', 'USER: <video> What food did I eat? ASSISTANT:', 'USER: <video> What did I put in the bowl? ASSISTANT:', 'USER: <video> where did i put my phone? ASSISTANT:', 'USER: <video> What tool did I use on the machine first? ASSISTANT:', 'USER: <video> What did I put in the dish? ASSISTANT:', 'USER: <video> Where was the egg before I picked it? ASSISTANT:', 'USER: <video> What did I pour in the mug? ASSISTANT:', 'USER: <video> Who did I talk to at the workshop? ASSISTANT:', 'USER: <video> what did l put in the bucket? ASSISTANT:', 'USER: <video> In what location did i see the car? ASSISTANT:', 'USER: <video> Where is round brush before picked? ASSISTANT:', 'USER: <video> Where did I wash the rice? ASSISTANT:', 'USER: <video> What did I take from the fridge? ASSISTANT:', 'USER: <video> where did  put the clothe before l hanged them? ASSISTANT:', 'USER: <video> How many doughs did i apply egg 

In [ ]:

inputs = []
for prompt, clip in zip(prompts, clips):
     inputs.append( processor(text=prompt, videos=clip, return_tensors="pt").to(model.device))


In [ ]:
generate_kwargs = {"max_new_tokens": 500}
generated_text = []
for input in inputs:
  generate_ids = model.generate(**input, **generate_kwargs)
  generated_text.append(processor.batch_decode(generate_ids, skip_special_tokens=True))#[0]

In [ ]:
print(generated_text)

[['USER:  Where is the soap? ASSISTANT: The soap is located in the sink.Ъ'], ['USER:  What vegetables did I chop?  ASSISTANT: You chopped a head of lettuce and a piece of broccoli.Ћ'], ['USER:  What food did I eat? ASSISTANT: The person in the video ate a plate of food, which included a sandwich and a bowl of food.Ћ'], ['USER:  What did I put in the bowl? ASSISTANT: The man in the video is using a power tool to mix cement in a large bowl.Ћ'], ['USER:  where did i put my phone? ASSISTANT: The person holding the cell phone is standing in a kitchen.Ъ'], ['USER:  What tool did I use on the machine first? ASSISTANT: The man used a wrench to unscrew the machine first.Ъ'], ['USER:  What did I put in the dish? ASSISTANT: The person in the video is seen putting food into a dish.Ъ'], ['USER:  Where was the egg before I picked it? ASSISTANT: The egg was in the refrigerator before you picked it.Ъ'], ['USER:  What did I pour in the mug? ASSISTANT: You poured coffee into the mug.Ъ'], ['USER:  Who di

In [ ]:
import re

cleaned_text = [[re.sub(r'[^\x00-\x7F]+', '', inner_text) for inner_text in text] for text in generated_text]

print(cleaned_text)
generated_text = cleaned_text


[['USER:  Where is the soap? ASSISTANT: The soap is located in the sink.'], ['USER:  What vegetables did I chop?  ASSISTANT: You chopped a head of lettuce and a piece of broccoli.'], ['USER:  What food did I eat? ASSISTANT: The person in the video ate a plate of food, which included a sandwich and a bowl of food.'], ['USER:  What did I put in the bowl? ASSISTANT: The man in the video is using a power tool to mix cement in a large bowl.'], ['USER:  where did i put my phone? ASSISTANT: The person holding the cell phone is standing in a kitchen.'], ['USER:  What tool did I use on the machine first? ASSISTANT: The man used a wrench to unscrew the machine first.'], ['USER:  What did I put in the dish? ASSISTANT: The person in the video is seen putting food into a dish.'], ['USER:  Where was the egg before I picked it? ASSISTANT: The egg was in the refrigerator before you picked it.'], ['USER:  What did I pour in the mug? ASSISTANT: You poured coffee into the mug.'], ['USER:  Who did I talk 

In [ ]:
import os
import json


def format_output(outputs, video_paths, output_json_path="/content/predictions_video_LLaVa.json"):
    formatted = ""
    counter = 1
    json_results = []

    for response_group, video_path in zip(outputs, video_paths):

        video_name = os.path.basename(video_path)

        for response in response_group:
            try:

                user_part, assistant_part = response.split("ASSISTANT:")
                user_question = user_part.replace("USER:", "").strip()
                assistant_answer = assistant_part.strip()


                formatted += (
                    f"---\n\n**{counter}. VIDEO:** {video_name}\n\n"
                    f"**USER:**\n{user_question}\n\n"
                    f"**ASSISTANT:**\n{assistant_answer}\n\n"
                )


                json_results.append({
                    "id": counter,
                    "video_name": video_name,
                    "user_question": user_question,
                    "assistant_answer": assistant_answer
                })

                counter += 1
            except ValueError:

                formatted += (
                    f"---\n\n**{counter}. VIDEO:** {video_name}\n\n"
                    f"**ERROR:**\nResponse format invalid: {response}\n\n"
                )

                json_results.append({
                    "id": counter,
                    "video_name": video_name,
                    "error": f"Response format invalid: {response}"
                })

                counter += 1


    with open(output_json_path, "w") as json_file:
        json.dump(json_results, json_file, indent=4)

    return formatted

formatted_output = format_output(generated_text, ordered_videos, output_json_path="/content/predictions_video_LLaVa.json")


print(formatted_output)


---

**1. VIDEO:** 5726971c-b3cc-43ed-8071-f6ee143e417d_0_14.mp4

**USER:**
Where is the soap?

**ASSISTANT:**
The soap is located in the sink.

---

**2. VIDEO:** 5d531ac1-010a-4e67-ba1a-96e485b14968_0_37.mp4

**USER:**
What vegetables did I chop?

**ASSISTANT:**
You chopped a head of lettuce and a piece of broccoli.

---

**3. VIDEO:** 1731de62-b1b9-4b84-bd55-412cd67e9b3c_0_18.mp4

**USER:**
What food did I eat?

**ASSISTANT:**
The person in the video ate a plate of food, which included a sandwich and a bowl of food.

---

**4. VIDEO:** 9093c341-5f95-4402-a2dd-6a84aa42bf1f_0_45.mp4

**USER:**
What did I put in the bowl?

**ASSISTANT:**
The man in the video is using a power tool to mix cement in a large bowl.

---

**5. VIDEO:** ab094ea2-9251-4f10-945b-c2ab00c5282e_0_7.mp4

**USER:**
where did i put my phone?

**ASSISTANT:**
The person holding the cell phone is standing in a kitchen.

---

**6. VIDEO:** b8654118-84a4-4167-83c9-f268cc15f7b2_0_26.mp4

**USER:**
What tool did I use on th

In [3]:

!pip install evaluate
!pip install rouge_score
!pip install sacrebleu
!pip install bert-score
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename

In [4]:
import json
import logging
import evaluate
from bert_score import score
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score
nltk.download('punkt_tab')
nltk.download('wordnet')


def sacre_bleu_score(candidate, references):
    sacrebleu = evaluate.load("sacrebleu")
    result = sacrebleu.compute(predictions=[candidate], references=[[ref] for ref in references])
    return result["score"]

def calculate_rouge_score(candidate, references):
    rouge_metric = evaluate.load("rouge")
    return rouge_metric.compute(predictions=[candidate], references=[references])

def bertscore_score(candidate, references):

    logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
    P, R, F1 = score([candidate], references, lang='en')
    return F1.item()
def meteor_score_function(candidate, references):

    candidate_tokens = word_tokenize(candidate.lower())
    references_tokens = [word_tokenize(ref.lower()) for ref in references]

    return meteor_score(references_tokens, candidate_tokens)


input_file = "/content/predictions_video_LLaVa.json"
output_file = "/content/output_with_metrics_video_LLava.json"
ground_truth_file = "/content/ground_truth.json"

with open(input_file, "r") as f:
    data = json.load(f)

with open(ground_truth_file, "r") as f:
    ground_truth_data = json.load(f)


video_name_to_ground_truth = {item["video_name"]: item["ground_truth"] for item in ground_truth_data}


for item in data:
    video_name = item["video_name"]

    if video_name in video_name_to_ground_truth:
        ground_truth = video_name_to_ground_truth[video_name]

        candidate = item["assistant_answer"]
        sacrebleu_result = sacre_bleu_score(candidate, [ground_truth])
        rouge_result = calculate_rouge_score(candidate, [ground_truth])
        bertscore_result = bertscore_score(candidate, [ground_truth])
        meteor_result = meteor_score_function(candidate, [ground_truth])


        item["ground_truth"] = ground_truth
        item["metrics"] = {
            "sacrebleu": sacrebleu_result,
            "rouge1": rouge_result["rouge1"],
            "rouge2": rouge_result["rouge2"],
            "rougeL": rouge_result["rougeL"],
            "bertscore": bertscore_result,
            "meteor": meteor_result,
        }


with open(output_file, "w") as f:
    json.dump(data, f, indent=4)

print(f"File JSON con metriche salvato in: {output_file}")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

File JSON con metriche salvato in: /content/output_with_metrics_video_LLava.json


In [5]:
import json
import matplotlib.pyplot as plt
import statistics


def calculate_statistics(data):
    metrics = ["sacrebleu", "rouge1", "rouge2", "rougeL", "bertscore", "meteor"]
    stats = {}

    for metric in metrics:
        values = [item["metrics"][metric] for item in data]
        stats[metric] = {
            "mean": sum(values) / len(values),
            "min": min(values),
            "max": max(values),
            "median": statistics.median(values),
            "std_dev": statistics.stdev(values) if len(values) > 1 else 0
        }
    return stats


def generate_scatter_plots(data, stats, output_dir="plots"):
    import os
    os.makedirs(output_dir, exist_ok=True)

    colors = {
        "sacrebleu": "blue",
        "rouge1": "green",
        "rouge2": "orange",
        "rougeL": "purple",
        "bertscore": "red",
        "meteor": "cyan"
    }

    metrics = ["sacrebleu", "rouge1", "rouge2", "rougeL", "bertscore", "meteor"]
    for metric in metrics:
        values = [item["metrics"][metric] for item in data]
        indices = list(range(1, len(values) + 1))
        mean_value = stats[metric]["mean"]

        plt.figure()
        plt.scatter(indices, values, alpha=0.7, color=colors[metric], label="Values")
        plt.axhline(y=mean_value, color='red', linestyle='--', label=f"Mean = {mean_value:.2f}")
        plt.title(f"Scatter Plot for {metric}")
        plt.xlabel("Sample Index")
        plt.ylabel(metric)


        if metric == "sacrebleu":
            plt.ylim(0, 100)
        else:
            plt.ylim(0, 1)
        plt.legend()
        plt.grid(True)
        plt.savefig(f"{output_dir}/{metric}_scatter_plot_with_mean_median_std.png")
        plt.close()


input_file = "/content/output_with_metrics_video_LLava.json"
stats_output_file = "/content/statistics.json"

with open(input_file, "r") as f:
    data = json.load(f)

stats = calculate_statistics(data)

with open(stats_output_file, "w") as f:
    json.dump(stats, f, indent=4)

print(f"Statistiche salvate in: {stats_output_file}")

generate_scatter_plots(data, stats)
print("Scatter plot generati nella cartella 'plots'.")


Statistiche salvate in: /content/statistics.json
Scatter plot generati nella cartella 'plots'.
